In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# 88:12 split

In [125]:
training_df = pd.read_csv('three_class_dataset.csv', index_col=0)
X = training_df[['headline_polarity', 'convo_polarity', 'forwardEps','bookValue', 'heldPercentInstitutions', 
        'shortRatio', 'shortPercentOfFloat']]
y = training_df['Prediction']

encoder = LabelEncoder()
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [126]:
from tensorflow.keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Dense(64, activation='softmax', input_shape=(7,)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = model.fit(
    X_train, y_train,
    batch_size=16,
    epochs=20,
    verbose=0,
    validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.38375788927078247
Test accuracy: 0.8901734352111816


# 75:25 split

In [144]:
training_df = pd.read_csv('three_class_dataset.csv', index_col=0)
X = training_df[['headline_polarity', 'convo_polarity', 'forwardEps','bookValue', 'heldPercentInstitutions', 
        'shortRatio', 'shortPercentOfFloat']]
y = training_df['Prediction']

encoder = LabelEncoder()
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

indexes = training_df[training_df['Prediction'] == 'Buy'].sample(frac=0.8).index
training_df = training_df.drop(indexes)

In [145]:
training_df['Prediction'].value_counts()

Buy     770
Hold    410
Sell     66
Name: Prediction, dtype: int64

In [146]:
from tensorflow.keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Dense(64, activation='softmax', input_shape=(7,)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = model.fit(
    X_train, y_train,
    batch_size=16,
    epochs=20,
    verbose=0,
    validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.38415589928627014
Test accuracy: 0.8901734352111816
